# **ReAct (Reasoning and Acting) Agent:**


The **ReAct agent** combines **reasoning** (verbal thought generation) and **acting** (executing task-specific actions) to solve tasks iteratively. Unlike traditional methods that separate reasoning and tool usage, ReAct tightly integrates these two steps, enabling more accurate and grounded decision-making.<br><br>

**Working Principal:**<br>
1. **Reasoning:** The model generates logical steps (thoughts) to plan actions.
2. **Action:** The model interacts with tools (e.g., search APIs) to gather external information.
3. **Observation:** Results are processed, and reasoning continues based on new insights.
4. **Iteration:** This cycle repeats until the task is solved.



In [ ]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4 boto3
%pip install --upgrade --quiet langchain-aws wikipedia youtube-search tavily-python

## **Load LLM:**

In [30]:
# Load the Tokens:

from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

print(llm.invoke("hi").content)

Hi there! How can I help you today?



## **Load Embeddings from HF:**

In [ ]:
%pip install --upgrade tokenizers

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **Vector Store:**

In [8]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [10]:
# retriever:

retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))

In [ ]:
retriever.get_relevant_documents("what is langchain?")

## **Implementation of ReAct Agent:**

### **Example 01: Simple implementation**

In [12]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [14]:
# initialized the Tool:

tools = [TavilySearchResults(max_results=5)]

In [15]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [16]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [17]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
Question: what is LangChain?
Thought: I need to search for information about LangChain.  I can use the tavily_search_results_json tool for this.
Action: tavily_search_results_json
Action Input: "What is LangChain?"[{'url': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'content': 'Data Structures and Algorithms\nML & Data Science\nWeb Development\nLanguages\nInterview Corner\nCS Subjects\nJobs\nPractice\nContests\nIntroduction to Langchain\nLangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). LangChain follows a general pipeline where a user asks a question to the language model where the vector representation of the question is used to do a similarity search in the vector database and the relevant information is fetched from the vector database and the response is later fed to the language model. LangChain Key Concepts:\nThe main properties of LangChain Framework

{'input': 'what is LangChain?',
 'output': "LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It allows developers to connect LLMs to other data sources and computational tools, enabling the development of more complex and context-aware applications.  Key features and functionalities include: chains (sequences of calls to LLMs or other utilities), agents (dynamically select which tools to use based on user input), indexes (structure documents for LLMs to use), and memory (persist application state between runs of a chain).  It's used to build applications like chatbots, question-answering systems, and summarization tools."}

### **Example 02: From Databse**

In [32]:
from langchain.tools import tool
from langchain import hub

In [33]:
# Custom tool for the Agent
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  employees_name = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}

  return employees_name.get(name, "Employee not found")



# Custom tool for the Agent
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")


# Get the Tools:
tools = [get_employee_salary, get_employee_id]

In [34]:
# Define the Prompt:
prompt = hub.pull("hwchase17/react")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [35]:
# Create Agent:
agent = create_react_agent(llm, tools, prompt)

In [36]:
# Call the Agent Executor:

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
agent_executor.invoke({"input":"What is the Salary and Employee Id of Fiona?"})

### **Example 03: Use Chat History**

#### **Create Memory:**

In [63]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [64]:
# store the session information in message_history:

message_history = ChatMessageHistory()
message_history

InMemoryChatMessageHistory(messages=[])

#### **Create Agent:**

In [65]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [66]:
# Create RAG Tool:

from langchain.tools.retriever import create_retriever_tool


retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="langsmith_search",
    description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [67]:
tools = [TavilySearchResults(max_results=1), retriever_tool]

In [68]:
# Define The Prompt:

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative res

In [69]:
# Construct the ReAct agent:

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [70]:
agent_executor.invoke(
    {
        "input": "what is langchain?",
        "chat_history": message_history.messages
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: What is LangChain?LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
LangSmith is a standalone platform that can be used on it's own no matter how you're creating your LLM applicatons.
In this tutorial, we'll walk you though logging your first trace in LangSmith using the LangSmith SDK and running an evaluation to measure the performance of your application. This example uses the OpenAI API, however you can use your provider of choice.
1. Install LangSmith‚Äã
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key‚Äã
To create an API key head to the Settings page. Then click Create API Ke

{'input': 'what is langchain?',
 'chat_history': [],
 'output': 'LangChain is a framework designed for developing applications powered by language models. It offers a streamlined process for connecting a language model to various data sources, enabling tasks like chatting with documents, querying APIs, and extracting structured information.  It also simplifies the management of prompts and chains of prompts, making it easier to create complex language model workflows.  While LangSmith integrates with LangChain for enhanced observability and evaluation, they are distinct entities.  LangChain focuses on building the applications, while LangSmith helps monitor and improve their performance.'}

#### **Add Conversation into Memory Dynamically using Session:**

In [71]:
# Agent Executor with Memory:

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

#### **Start Interaction:**

In [72]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hi Dibyendu! I understand you're an AI/ML Engineer and currently looking for a job. That can be a tough situation.  I'm here to help if I can. Is there anything specific you'd like to talk about or get assistance with?  For example, are you looking for advice on your resume, interview tips, or help finding job postings?


> Finished chain.


{'input': "hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job",
 'chat_history': [],
 'output': "Hi Dibyendu! I understand you're an AI/ML Engineer and currently looking for a job. That can be a tough situation.  I'm here to help if I can. Is there anything specific you'd like to talk about or get assistance with?  For example, are you looking for advice on your resume, interview tips, or help finding job postings?"}

In [73]:
agent_with_chat_history.invoke(
    {"input": "What is LangChain?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: What is LangChain?LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
LangSmith is a standalone platform that can be used on it's own no matter how you're creating your LLM applicatons.
In this tutorial, we'll walk you though logging your first trace in LangSmith using the LangSmith SDK and running an evaluation to measure the performance of your application. This example uses the OpenAI API, however you can use your provider of choice.
1. Install LangSmith‚Äã
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key‚Äã
To create an API key head to the Settings page. Then click Create API Ke

{'input': 'What is LangChain?',
 'chat_history': [HumanMessage(content="hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi Dibyendu! I understand you're an AI/ML Engineer and currently looking for a job. That can be a tough situation.  I'm here to help if I can. Is there anything specific you'd like to talk about or get assistance with?  For example, are you looking for advice on your resume, interview tips, or help finding job postings?", additional_kwargs={}, response_metadata={})],
 'output': 'LangChain is a framework designed for developing applications powered by language models. It offers a streamlined process for connecting a language model to various data sources, enabling tasks like chatting with documents, querying APIs, and extracting structured data.  It also provides modules for managing prompts, chains for sequencing calls to LLMs or other utilities, and agents to decide which act

In [74]:
agent_with_chat_history.invoke(
    {"input": "Who is Dibyendu?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: You told me you are Dibyendu, an AI/ML Engineer looking for a job.  Is there anything else I can tell you about yourself based on what you've shared with me?


> Finished chain.


{'input': 'Who is Dibyendu?',
 'chat_history': [HumanMessage(content="hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi Dibyendu! I understand you're an AI/ML Engineer and currently looking for a job. That can be a tough situation.  I'm here to help if I can. Is there anything specific you'd like to talk about or get assistance with?  For example, are you looking for advice on your resume, interview tips, or help finding job postings?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is LangChain?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='LangChain is a framework designed for developing applications powered by language models. It offers a streamlined process for connecting a language model to various data sources, enabling tasks like chatting with documents, querying APIs, and extracting structured data.  It also provides modules for managing

In [77]:
message_history.messages

[HumanMessage(content="hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi Dibyendu! I understand you're an AI/ML Engineer and currently looking for a job. That can be a tough situation.  I'm here to help if I can. Is there anything specific you'd like to talk about or get assistance with?  For example, are you looking for advice on your resume, interview tips, or help finding job postings?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is LangChain?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain is a framework designed for developing applications powered by language models. It offers a streamlined process for connecting a language model to various data sources, enabling tasks like chatting with documents, querying APIs, and extracting structured data.  It also provides modules for managing prompts, chains for sequencing calls to LLMs or o